### Import libraries and data

In [ ]:
import pandas as pd
import csv
import re
import numpy as np

In [ ]:
vols = pd.read_csv("volumes.csv")
vols = vols.drop("Unnamed: 0", axis=1)

In [ ]:
temps = pd.read_csv("temperatures.csv")
temps = temps.drop("Unnamed: 0", axis=1)

In [ ]:
unitformula = pd.read_csv("scaledformulae.csv")
unitformula = unitformula.drop("Unnamed: 0", axis=1)

### Setting models

In [ ]:
from sklearn import linear_model, svm, kernel_ridge
regVols = linear_model.LinearRegression()
regAlpha = linear_model.LinearRegression()
#regVols = linear_model.ElasticNet(positive = True)

### Percentage function

In [ ]:
def percCalc (row):
    predictedvolume = row["sum"]
    vols = row["Volume"]
    percentage = predictedvolume / vols
    if (percentage >=0.95) and (percentage <=1.05):
        return 1
    else:
        return 0

### Loop algorithm

In [ ]:
alpha = 0.00
x2=pd.DataFrame()
keeptemps = temps
keepvols = vols
elements = unitformula.columns.values
unitformula["Keep"] = 1
dropidx = 1
for step in range(1,3):
    print step, unitformula["Keep"].sum()
    regVols.fit(unitformula[unitformula.Keep == 1], keepvols.div(keeptemps.Temperature*alpha+1, axis="index"))
    w = regVols.coef_[0]
    for line in zip(elements, w):
        x2[line[0]] = unitformula[line[0]].apply(lambda x: x*line[1])
    x2["sum"] = x2.sum(axis=1)
    if dropidx == 1:
        vol2 = keepvols.div(x2["sum"], axis="index") - 1
    else:
        vol2 = keepvols.div(x2["sum"].drop(x2.index[[dropidx]]), axis="index") - 1
    regAlpha.fit(keeptemps, vol2)
    alpha = regAlpha.coef_[0][0]
    unitformula["Keep"] = pd.concat([x2,vols], axis=1).apply(percCalc, axis=1)
    print "Element average volume (cubic angstroms)", zip(elements,w)
    print "Alpha Value: ", alpha
    dropidx = unitformula['Keep'][unitformula["Keep"] != 1].index.tolist()
    keeptemps = temps.drop(temps.index[[dropidx]])
    keepvols = vols.drop(vols.index[[dropidx]])

This is just a continuation of the previous notebook

In [ ]:
print unitformula[unitformula.Keep == 1]

The "Keep" column is printed to view

In [3]:
x2.loc[x2["sum"] == 0]

The location of predicted volumes which were 0 are outputted, this was used just to check if there were any predictions of 0 that would cause a division error